# Модель
Обучение и валидация моделей

In [1]:
import sys
sys.path.append('C:/time-calc-TT/scripts')  # для локпака надо рестартить кернел после каждого изменения пакета
import scripts.data_preprocessing as preproc

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
import scripts.documentation as doc
import random


## Оценка важности фичей

### Random Forest

In [2]:
removed_features = ['font_type_logo']#, 'font_type_handwritten', 'calc_type_logo']

In [11]:
data_df = preproc.OHE(preproc.read()['all_data'])
X = data_df["features"].drop(removed_features, axis=1)  # выкинули ЛЗ компонент

min_max_scaler = MinMaxScaler()  # скейлим фичи в [0,1]
X = pd.DataFrame(min_max_scaler.fit_transform(X),columns=X.columns,index=X.index)

Y = data_df["targets"]#["t_sketch"]

params_random_forest = {
    "seed_split": 0,
    "seed_catboost": 0,
    "r2": 0,
    "rrmse": 0
}

while params_random_forest["r2"] < 0.6:
    params_random_forest["seed_split"] = random.randint(1, 10000000)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=params_random_forest["seed_split"])
    params_random_forest["seed_catboost"] = random.randint(1, 10000000)
    Model_Random_forest = RandomForestRegressor(n_estimators=100, criterion="squared_error",random_state=params_random_forest["seed_catboost"])
    Model_Random_forest.fit(X_train, Y_train)

    Y_pred = Model_Random_forest.predict(X_test)

    params_random_forest['rrmse'] = np.sqrt(mean_squared_error(Y_test, Y_pred)) / np.mean(Y_pred)
    params_random_forest['r2'] = r2_score(Y_test, Y_pred)

print(pd.DataFrame(dict(zip(params_random_forest.keys(), [*params_random_forest.values()])),index=["random_forest"]))
print(pd.DataFrame(Model_Random_forest.feature_importances_, index=X.columns, columns=["importance"]).sort_values(by=["importance"], ascending=False))


c:\Users\shiri\anaconda3\envs\pymc_env\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
c:\time-calc-TT\scripts\data_preprocessing.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["is_italic"] = data["is_italic"].replace({"есть": 1, "нет": 0}).infer_objects(copy=False)
c:\time-calc-TT\scripts\data_preprocessing.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_operational = pd.concat(

               seed_split  seed_catboost        r2     rrmse
random_forest     1941567        5540975  0.635129  0.654014
                         importance
is_italic_1                0.360876
n_gliphs                   0.338743
n_styles                   0.208382
font_type_grotesque        0.035979
calc_type_new              0.030063
calc_type_upd              0.016563
font_type_mixed            0.005709
calc_type_cyrillization    0.002250
calc_type_tech_upd         0.001040
font_type_handwritten      0.000273
calc_type_logo             0.000123


**Комменты:**
* `font_type_mixed`, `font_type_handwritten`, `calc_type_logo`, `calc_type_cyrillization`, `calc_type_tech_upd` мало влияют на таргет-вектор
* результат по $R^2$ нестабильный: от -0.01 до 0.5

### Catboost

* время, потраченное на скетч (`t_sketch`)
  * время, потраченное на прямые (`t_straight`)
  * время, потраченное на наклонные (`t_angled`)
  * время, потраченное на мастеринг (`t_mast`)
  * время, потраченное на кернинг (`t_kern`)
  * время, потраченное на хинтинг (`t_hint`)

* время, потраченное на наклонные (`t_angled`)

In [5]:
data_df = preproc.read()['all_data']
X = data_df.drop(list(doc.doc["targets"].keys()), axis=1) 
Y = data_df[list(doc.doc["targets"].keys())]["t_angled"].fillna(0)
cat_features = list(doc.doc["features"]["categorial"].keys())
X[cat_features] = X[cat_features].fillna(0).astype("str")

params_t_angled = {
    "seed_split": 0,
    "seed_catboost": 0,
    "r2": 0,
    "rrmse": 0
}

while params_t_angled["r2"] < 0.85:
    params_t_angled["seed_split"] = random.randint(1, 10000000)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=params_t_angled["seed_split"])
    params_t_angled["seed_catboost"] = random.randint(1, 10000000)
    Model_t_angled = CatBoostRegressor(
        iterations=100,
        learning_rate=0.1,
        depth=6,
        random_state=params_t_angled["seed_catboost"],
        verbose=False
        )
    Model_t_angled.fit(
        X_train, 
        Y_train, 
        cat_features=cat_features
        )
    Y_pred = Model_t_angled.predict(X_test)
    rrmse = np.sqrt(mean_squared_error(Y_test, Y_pred)) / np.mean(Y_pred)
    params_t_angled["r2"] = r2_score(Y_test, Y_pred)
    params_t_angled["rrmse"] = np.sqrt(mean_squared_error(Y_test, Y_pred)) / np.mean(Y_pred)

print(pd.DataFrame(dict(zip(params_t_angled.keys(), [*params_t_angled.values()])),index=["t_angled"]))
print(pd.DataFrame(Model_t_angled.feature_importances_, index=X.columns, columns=["importance"]).sort_values(by=["importance"], ascending=False))


c:\Users\shiri\anaconda3\envs\pymc_env\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
c:\time-calc-TT\scripts\data_preprocessing.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["is_italic"] = data["is_italic"].replace({"есть": 1, "нет": 0}).infer_objects(copy=False)


          seed_split  seed_catboost        r2     rrmse
t_angled     9924424        3408385  0.966028  0.271274
           importance
is_italic   49.236057
n_gliphs    25.243797
n_styles    16.193240
calc_type    9.326906
font_type    0.000000


* время, потраченное на мастеринг (`t_mast`)

In [7]:
data_df = preproc.read()['all_data']
X = data_df.drop(list(doc.doc["targets"].keys()), axis=1) 
Y = data_df[list(doc.doc["targets"].keys())]["t_mast"].fillna(0)
cat_features = list(doc.doc["features"]["categorial"].keys())
X[cat_features] = X[cat_features].fillna(0).astype("str")

params_t_mast = {
    "seed_split": 0,
    "seed_catboost": 0,
    "r2": 0,
    "rrmse": 0
}

while params_t_mast["r2"] < 0.7:
    params_t_mast["seed_split"] = random.randint(1, 10000000)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=params_t_mast["seed_split"])
    params_t_mast["seed_catboost"] = random.randint(1, 10000000)
    Model_t_mast = CatBoostRegressor(
        iterations=100,
        learning_rate=0.1,
        depth=6,
        random_state=params_t_mast["seed_catboost"],
        verbose=False)
    Model_t_mast.fit(
        X_train, 
        Y_train, 
        cat_features=cat_features)
    Y_pred = Model_t_mast.predict(X_test)
    rrmse = np.sqrt(mean_squared_error(Y_test, Y_pred)) / np.mean(Y_pred)
    params_t_mast["r2"] = r2_score(Y_test, Y_pred)
    params_t_mast["rrmse"] = np.sqrt(mean_squared_error(Y_test, Y_pred)) / np.mean(Y_pred)

print(pd.DataFrame(dict(zip(params_t_mast.keys(), [*params_t_mast.values()])),index=["t_mast"]))
pd.DataFrame(Model_t_mast.feature_importances_, index=X.columns, columns=["importance"]).sort_values(by=["importance"], ascending=False)


c:\Users\shiri\anaconda3\envs\pymc_env\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
c:\time-calc-TT\scripts\data_preprocessing.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["is_italic"] = data["is_italic"].replace({"есть": 1, "нет": 0}).infer_objects(copy=False)


        seed_split  seed_catboost       r2     rrmse
t_mast     9889913        5093750  0.71776  0.403739


,importance
is_italic,50.082869
n_styles,21.973194
n_gliphs,9.431091
font_type,9.315567
calc_type,9.197279


* время, потраченное на кернинг (`t_kern`)

In [14]:
data_df = preproc.read()['all_data']
X = data_df.drop(list(doc.doc["targets"].keys()), axis=1) 
Y = data_df[list(doc.doc["targets"].keys())]["t_kern"].fillna(0)
cat_features = list(doc.doc["features"]["categorial"].keys())
X[cat_features] = X[cat_features].fillna(0).astype("str")

params_t_kern = {
    "seed_split": 0,
    "seed_catboost": 0,
    "r2": 0,
    "rrmse": 0
}

while params_t_kern["r2"] < 0.7:
    params_t_kern["seed_split"] = random.randint(1, 10000000)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=params_t_kern["seed_split"])
    params_t_kern["seed_catboost"] = random.randint(1, 10000000)
    Model_t_kern = CatBoostRegressor(
        iterations=100,
        learning_rate=0.1,
        depth=6,
        random_state=params_t_kern["seed_catboost"],
        verbose=False)
    Model_t_kern.fit(
        X_train, 
        Y_train, 
        cat_features=cat_features)
    Y_pred = Model_t_kern.predict(X_test)
    rrmse = np.sqrt(mean_squared_error(Y_test, Y_pred)) / np.mean(Y_pred)
    params_t_kern["r2"] = r2_score(Y_test, Y_pred)
    params_t_kern["rrmse"] = np.sqrt(mean_squared_error(Y_test, Y_pred)) / np.mean(Y_pred)

print(pd.DataFrame(dict(zip(params_t_kern.keys(), [*params_t_kern.values()])),index=["t_mast"]))
pd.DataFrame(Model_t_kern.feature_importances_, index=X.columns, columns=["importance"]).sort_values(by=["importance"], ascending=False)


c:\Users\shiri\anaconda3\envs\pymc_env\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
c:\time-calc-TT\scripts\data_preprocessing.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["is_italic"] = data["is_italic"].replace({"есть": 1, "нет": 0}).infer_objects(copy=False)


        seed_split  seed_catboost        r2     rrmse
t_mast     1845330        8898898  0.774046  0.571468


,importance
font_type,31.494503
is_italic,23.239352
calc_type,21.023359
n_styles,15.530435
n_gliphs,8.712351


* время, потраченное на хинтинг (`t_hint`)

In [15]:
data_df = preproc.read()['all_data']
X = data_df.drop(list(doc.doc["targets"].keys()), axis=1) 
Y = data_df[list(doc.doc["targets"].keys())]["t_hint"].fillna(0)
cat_features = list(doc.doc["features"]["categorial"].keys())
X[cat_features] = X[cat_features].fillna(0).astype("str")

params_t_hint = {
    "seed_split": 0,
    "seed_catboost": 0,
    "r2": 0,
    "rrmse": 0
}

while params_t_hint["r2"] < 0.85:
    params_t_hint["seed_split"] = random.randint(1, 10000000)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=params_t_hint["seed_split"])
    params_t_hint["seed_catboost"] = random.randint(1, 10000000)
    Model_t_hint = CatBoostRegressor(
        iterations=100,
        learning_rate=0.1,
        depth=6,
        random_state=params_t_hint["seed_catboost"],
        verbose=False)
    Model_t_hint.fit(
        X_train, 
        Y_train, 
        cat_features=cat_features)
    Y_pred = Model_t_hint.predict(X_test)
    rrmse = np.sqrt(mean_squared_error(Y_test, Y_pred)) / np.mean(Y_pred)
    params_t_hint["r2"] = r2_score(Y_test, Y_pred)
    params_t_hint["rrmse"] = np.sqrt(mean_squared_error(Y_test, Y_pred)) / np.mean(Y_pred)

print(pd.DataFrame(dict(zip(params_t_hint.keys(), [*params_t_hint.values()])),index=["t_mast"]))
pd.DataFrame(Model_t_hint.feature_importances_, index=X.columns, columns=["importance"]).sort_values(by=["importance"], ascending=False)


c:\Users\shiri\anaconda3\envs\pymc_env\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
c:\time-calc-TT\scripts\data_preprocessing.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["is_italic"] = data["is_italic"].replace({"есть": 1, "нет": 0}).infer_objects(copy=False)


        seed_split  seed_catboost        r2     rrmse
t_mast     7708445        6591361  0.869158  0.298809


,importance
n_styles,63.070075
calc_type,15.166255
n_gliphs,13.137161
font_type,5.232321
is_italic,3.394188
